In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D,Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape


In [2]:
class  DCGAN:
    @staticmethod
    def build_generator(dim,depth,channels = 1,inputDim = 100, outputDim = 512):
        #where dim is the spatial dimension of the input volume (i.e., the width and height of the input image)
        #depth is the number of filters in the first layer of the generator(we will double the number of filters in each subsequent layer)
        
        model = Sequential()
        inputShape = (dim,dim,depth) #input shape of the generator
        chanDim = -1 #ChanDim is the channel dimension meaning that the data is in the format of (batch, height, width, channels)
                     #1 for grayscale images and 3 for RGB images
        
        
        #first set of FC => RELU => BN layers
        model.add(Dense(input_dim= inputDim,units=outputDim)) #fully connected layer where units is the number of nodes in the layer
        model.add(Activation("relu") ) #activation function relu
        model.add(BatchNormalization()) #batch normalization meaning that we normalize the activations of the network at each layer divided into mini-batches
        
        #second set of FC => RELU => BN layers 
        #this time the number of FC nodes to be reshaped into a volume becasue we are going to apply a convolutional transpose layer where input_dim < output_dim
        
        model.add(Dense(dim*dim*depth)) #fully connected layer has output of dim*dim*depth
        model.add(Activation("relu")) #activation function relu
        model.add(BatchNormalization()) #batch normalization    
        
        #reshape the volume to have the desired spatial dimensions
        model.add(Reshape(inputShape)) #reshape the volume to have the desired spatial dimensions
        model.add(Conv2DTranspose(32,(5,5),strides=(2,2),padding="same")) #convolutional transpose layer where 32 is the number of filters and (5,5) is the size of the filter and strides is the step size of the filter
        model.add(Activation("relu")) #activation function relu
        model.add(BatchNormalization(axis=chanDim)) #batch normalization where chanDim is -1 means it takes the last dimension of the input
        
        #apply another convolutional transpose layer
        model.add(Conv2DTranspose(channels,(5,5),strides=(2,2),padding="same")) #convolutional transpose layer where channels is the number of filters and (5,5) is the size of the filter and strides is the step size of the filter
        model.add(Activation("tanh")) #activation function tanh
        
        
        return model
    
    @staticmethod
    def build_discriminator(width,height,depth,alpha=0.2):
        #where width and height are the spatial dimensions of the input volume
        #depth is the number of filters in the first layer of the discriminator
        #alpha is the slope of the leaky ReLU activation function in simple terms it is the learning rate of the activation function
        
        model = Sequential()
        inputShape = (height,width,depth)
        
        #first set of CONV => RELU layers
        model.add(Conv2D(32,(5,5),padding="same",input_shape=inputShape))
        model.add(LeakyReLU(alpha=alpha))
        
        
        #second set of CONV => RELU layers  
        model.add(Conv2D(64,(5,5),strides = (2,2),padding="same"))
        model.add(LeakyReLU(alpha=alpha))
        
        #third set Flatten => FC layer (fully connected layer)
        model.add(Flatten()) #flatten the volume into a vector as the input to the fully connected layer in column major order
        model.add(Dense(512)) #512 is the number of nodes in the layer
        model.add(LeakyReLU(alpha=alpha))
        
        #output layer
        model.add(Dense(1))
        model.add(Activation("sigmoid"))
        
        return model
        
        


In [3]:
model= DCGAN.build_generator(7,64)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               51712     
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 3136)              1608768   
                                                                 
 activation_1 (Activation)   (None, 3136)              0         
                                                                 
 batch_normalization_1 (Batc  (None, 3136)             12544     
 hNormalization)                                        

In [4]:
model = DCGAN.build_discriminator(28,28,1)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 28, 28, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        51264     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense_2 (Dense)             (None, 512)               6423040   
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 512)              